In [1]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
df = pd.read_excel('Data_InCarMusic.xlsx')
df.head()

,UserID,ItemID,Rating,DrivingStyle,landscape,mood,naturalphenomena,RoadType,sleepiness,trafficConditions,weather
0,1001,715,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sunny
1,1001,267,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sunny
2,1001,294,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sunny
3,1001,259,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,snowing
4,1001,674,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rainy


In [21]:
df = df[df[' Rating'] > 3]
df.head()

,UserID,ItemID,Rating,DrivingStyle,landscape,mood,naturalphenomena,RoadType,sleepiness,trafficConditions,weather
1,1001,267,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sunny
3,1001,259,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,snowing
6,1001,259,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cloudy
8,1001,285,4,NaN,NaN,NaN,NaN,NaN,NaN,lots of cars,NaN
11,1001,694,4,NaN,NaN,NaN,NaN,NaN,awake,NaN,NaN


In [22]:
df = df.fillna('x')

In [23]:
df['bag_of_words'] = df[['DrivingStyle', 'landscape', 'mood','naturalphenomena ', 'RoadType', 'sleepiness', 'trafficConditions', 'weather']].apply(lambda x: ' '.join(x.astype(str)), axis=1)

In [24]:
df['bag_of_words'] = df['bag_of_words'].apply(lambda x: str(x).replace('x','')).apply(lambda x: str(x).replace(' ',''))

df = df[['UserID','ItemID',' Rating','bag_of_words']]

In [25]:
df_vector = df.groupby(['ItemID']).agg(lambda x: ' '.join(set(x))).reset_index()

In [26]:
df_vector.head()

,ItemID,bag_of_words
0,248,happy cloudy freeroad lotsofcars awake highwa...
1,249,cloudy lotsofcars freeroad awake sad sunny sl...
2,250,snowing cloudy freeroad rainy city awake high...
3,251,happy snowing cloudy rainy lotsofcars coastli...
4,252,snowing freeroad lotsofcars sleepy highway co...


In [27]:
#In order to detect similarities between songs, we need to vectorize the tags in each
count = CountVectorizer()
count_matrix = count.fit_transform(df_vector['bag_of_words'])

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [39]:
cosine_sim


array([[1.        , 0.5976143 , 0.52414242, ..., 0.26726124, 0.18898224,
        0.        ],
       [0.5976143 , 1.        , 0.43852901, ..., 0.4472136 , 0.        ,
        0.        ],
       [0.52414242, 0.43852901, 1.        , ..., 0.2941742 , 0.2773501 ,
        0.32025631],
       ...,
       [0.26726124, 0.4472136 , 0.2941742 , ..., 1.        , 0.1767767 ,
        0.        ],
       [0.18898224, 0.        , 0.2773501 , ..., 0.1767767 , 1.        ,
        0.        ],
       [0.        , 0.        , 0.32025631, ..., 0.        , 0.        ,
        1.        ]])

In [63]:
indices = df_vector['ItemID']

def recommendations(ItemID, cosine_sim = cosine_sim):
    recommended_songs = []
    idx = indices[indices == ItemID].index[0]

    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    
    top_5_indexes = list(score_series.iloc[1:6].index)
    for i in top_5_indexes:
        recommended_songs.append(list(df_vector.index)[i])
        
    return recommended_songs

In [69]:
rec = recommendations(ItemID=715)

In [70]:
df_vector.iloc[rec]

,ItemID,bag_of_words
106,735,lazy night rainy
89,718,rainy freeroad sad urban
71,698,lazy night coastline
116,746,happy rainy cloudy coastline awake sad sunny ...
40,289,happy rainy coastline sad sportdriving night ...
